In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import os
from scipy import stats as ss
from sklearn import linear_model
import unicodedata
import unidecode
import matplotlib.pyplot as plt



In [153]:
! pip install unidecode

In [2]:

df_ventas = pd.read_pickle(f'../data/copia/df_ventas.pkl')

In [4]:
pd.set_option('mode.chained_assignment','warn')

In [3]:
df_ventas.is_copy = False

In [5]:
df_ventas.head()

,Marca,Modelo,Version,Precio,Precio_financiado,Combustible,Anyo,Kilometros,Potencia,tipo_cambio,Vendedor_profesional,Vendedor,Provincia,Comunidad_Autonoma
0,SEAT,Toledo,SEAT Toledo 4p.,950,NaN,Diésel,2000.0,227000,NaN,Manual,False,0f4bb8455d27349b8273109b66a847f3,Navarra,Navarra
1,CITROEN,C1,CITROEN C1 PureTech 60KW 82CV Feel 5p.,6200,NaN,Gasolina,2017.0,50071,82.0,Manual,True,Autos Raymara,Tenerife,Canarias
2,FORD,Transit Connect,FORD Transit Connect Van 1.5 TDCi 100cv Ambien...,7851,7024.0,Diésel,2016.0,103000,100.0,Manual,True,Auto 96,Barcelona,Catalunya
3,VOLKSWAGEN,Caravelle,VOLKSWAGEN Caravelle Largo 2.0 TDI 140 Comfort...,19426,NaN,Diésel,2014.0,120000,140.0,Manual,True,Inniauto,Navarra,Navarra
4,FORD,Transit,FORD Transit 350 96kW L4 Ambiente Propulsion T...,22850,22800.0,Diésel,2017.0,107000,130.0,Manual,True,"Autofleet España,s.l",Sevilla,Andalucía


In [7]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marca                 49998 non-null  object 
 1   Modelo                49995 non-null  object 
 2   Version               50000 non-null  object 
 3   Precio                50000 non-null  int64  
 4   Precio_financiado     23563 non-null  float64
 5   Combustible           49954 non-null  object 
 6   Anyo                  49998 non-null  float64
 7   Kilometros            50000 non-null  int64  
 8   Potencia              41472 non-null  float64
 9   tipo_cambio           49889 non-null  object 
 10  Vendedor_profesional  50000 non-null  bool   
 11  Vendedor              50000 non-null  object 
 12  Provincia             49994 non-null  object 
 13  Comunidad_Autonoma    50000 non-null  object 
dtypes: bool(1), float64(3), int64(2), object(8)
memory usage: 5.0+ MB


In [8]:
df_ventas.corr()

,Precio,Precio_financiado,Anyo,Kilometros,Potencia,Vendedor_profesional
Precio,1.000000,0.997199,0.476886,-0.394514,0.693192,0.339699
Precio_financiado,0.997199,1.000000,0.329882,-0.326330,0.741984,NaN
Anyo,0.476886,0.329882,1.000000,-0.620422,0.027102,0.612190
Kilometros,-0.394514,-0.326330,-0.620422,1.000000,-0.012646,-0.447068
Potencia,0.693192,0.741984,0.027102,-0.012646,1.000000,0.003982
Vendedor_profesional,0.339699,NaN,0.612190,-0.447068,0.003982,1.000000


In [4]:
df_ventas.loc[df_ventas.Marca.isnull(),['Version','Modelo','Combustible','Potencia','Precio','Provincia']]

,Version,Modelo,Combustible,Potencia,Precio,Provincia
5659,D2S 27kW 3p.,D2S,Eléctrico,37.0,21495,Burgos
47381,D2S 27kW 3p.,D2S,Eléctrico,37.0,21495,Burgos


### Asigno el nombre de la Marca a los vehículos con Modelo D2S

In [5]:
df_ventas['Marca'][df_ventas.Marca.isnull()] = 'Invicta Electric';

C:\Users\saula\AppData\Local\Temp\ipykernel_19380\3452234145.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas['Marca'][df_ventas.Marca.isnull()] = 'Invicta Electric';


In [11]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marca                 50000 non-null  object 
 1   Modelo                49995 non-null  object 
 2   Version               50000 non-null  object 
 3   Precio                50000 non-null  int64  
 4   Precio_financiado     23563 non-null  float64
 5   Combustible           49954 non-null  object 
 6   Anyo                  49998 non-null  float64
 7   Kilometros            50000 non-null  int64  
 8   Potencia              41472 non-null  float64
 9   tipo_cambio           49889 non-null  object 
 10  Vendedor_profesional  50000 non-null  bool   
 11  Vendedor              50000 non-null  object 
 12  Provincia             49994 non-null  object 
 13  Comunidad_Autonoma    50000 non-null  object 
dtypes: bool(1), float64(3), int64(2), object(8)
memory usage: 5.0+ MB


### Nulos Combustible

In [12]:
df_ventas.loc[df_ventas.Combustible.isnull(),['Version','Modelo','Combustible','Potencia','Precio']]

,Version,Modelo,Combustible,Potencia,Precio
1921,BMW Serie 6 2p.,Serie 6,NaN,NaN,17000
1942,FIAT Tipo 5p.,Tipo,NaN,NaN,13900
3293,ALFA ROMEO 147 3p.,147,NaN,NaN,4500
3896,RENAULT,Kangoo,NaN,NaN,7000
6022,MERCEDES-BENZ Clase C 3p.,Clase C,NaN,NaN,2500
6213,HYUNDAI IONIQ 5p.,IONIQ,NaN,NaN,19500
7360,SEAT Cordoba 5p.,Córdoba,NaN,NaN,1700
7660,TOYOTA Auris 5p.,Auris,NaN,NaN,14500
9647,FORD Focus 5p.,Focus,NaN,NaN,23900
12732,HYUNDAI IONIQ 5p.,IONIQ,NaN,NaN,17900


In [13]:
df_ventas.Combustible[df_ventas.Modelo == '147']

816      Gasolina
1314       Diésel
2402       Diésel
2477       Diésel
3293          NaN
4010       Diésel
5023       Diésel
6503       Diésel
7023     Gasolina
7145     Gasolina
7426       Diésel
10679      Diésel
11048    Gasolina
11426      Diésel
11907      Diésel
12558      Diésel
13409      Diésel
14025    Gasolina
15204      Diésel
16773      Diésel
16993      Diésel
17098      Diésel
22248    Gasolina
24751      Diésel
24786    Gasolina
25125      Diésel
25483    Gasolina
25491      Diésel
27363      Diésel
28013    Gasolina
30742      Diésel
32550    Gasolina
32995      Diésel
35388      Diésel
35912      Diésel
38246      Diésel
39248      Diésel
39319    Gasolina
39888      Diésel
44074    Gasolina
44268    Gasolina
44449    Gasolina
45294    Gasolina
47600    Gasolina
48921      Diésel
49699      Diésel
Name: Combustible, dtype: object

In [14]:
df_ventas.Combustible[df_ventas.Modelo == 'Auris']

172       Híbrido
215       Híbrido
390       Híbrido
433        Diésel
539       Híbrido
           ...   
49413     Híbrido
49568     Híbrido
49711      Diésel
49716     Híbrido
49773    Gasolina
Name: Combustible, Length: 335, dtype: object

No se pueden rellenar los nulos de Combustible por lo que se desechan

### Nulos Anyo

In [15]:
df_ventas.loc[df_ventas.Anyo.isnull(),['Version','Modelo','Combustible','Potencia','Precio','Kilometros']]

,Version,Modelo,Combustible,Potencia,Precio,Kilometros
31025,KIA Stonic 1.0 TGDi 74kW 100CV Concept 5p.,Stonic,Gasolina,100.0,16850,1
42075,SEAT Arona 1.0 TSI 85kW 115CV Xcellence Ecomot...,Arona,Gasolina,115.0,22500,1


KIA --> 2019 (buscado en web y en la propia lista de coches)\
Arona --> 2020 (buscado en web y por caracteristicas de los coches de la lista)

In [16]:
df_ventas.loc[df_ventas.Modelo == 'Arona',['Version','Modelo','Combustible','Potencia','Precio','Kilometros','Anyo']]

,Version,Modelo,Combustible,Potencia,Precio,Kilometros,Anyo
97,SEAT Arona 1.6 TDI 85kW 115CV FR Editio Eco 5p.,Arona,Diésel,115.0,19900,17788,2019.0
629,SEAT Arona 1.0 TSI 85kW 115CV FR Ecomotive 5p.,Arona,Gasolina,115.0,18900,4000,2019.0
760,SEAT Arona 1.0 TSI 85kW 115CV FR Ecomotive 5p.,Arona,Gasolina,115.0,15900,50000,2019.0
1794,SEAT Arona 1.6 TDI 70kW 95CV Style Ecomotive 5p.,Arona,Diésel,95.0,15799,54607,2018.0
2423,SEAT Arona 1.0 TSI 85kW 115CV Style Edition Ec...,Arona,Gasolina,115.0,16490,5554,2019.0
...,...,...,...,...,...,...,...
49059,SEAT Arona 1.6 TDI 70kW 95CV Xcellence Ecomoti...,Arona,Diésel,95.0,18500,25851,2019.0
49081,SEAT Arona 1.0 TSI 70kW 95CV Style Go Ecomotiv...,Arona,Gasolina,95.0,18380,0,2021.0
49548,SEAT Arona 1.6 TDI 85kW 115CV Style Edition Ec...,Arona,Diésel,115.0,16990,6427,2019.0
49723,SEAT Arona 1.0 TSI 70kW 95CV Style Go Eco 5p.,Arona,Gasolina,95.0,19025,1,2020.0


In [17]:
df_ventas.Anyo[31025] = 2019
df_ventas.Anyo[42075] = 2020

C:\Users\saula\AppData\Local\Temp\ipykernel_20272\2032288916.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas.Anyo[31025] = 2019
C:\Users\saula\AppData\Local\Temp\ipykernel_20272\2032288916.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas.Anyo[42075] = 2020


In [18]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marca                 50000 non-null  object 
 1   Modelo                49995 non-null  object 
 2   Version               50000 non-null  object 
 3   Precio                50000 non-null  int64  
 4   Precio_financiado     23563 non-null  float64
 5   Combustible           49954 non-null  object 
 6   Anyo                  50000 non-null  float64
 7   Kilometros            50000 non-null  int64  
 8   Potencia              41472 non-null  float64
 9   tipo_cambio           49889 non-null  object 
 10  Vendedor_profesional  50000 non-null  bool   
 11  Vendedor              50000 non-null  object 
 12  Provincia             49994 non-null  object 
 13  Comunidad_Autonoma    50000 non-null  object 
dtypes: bool(1), float64(3), int64(2), object(8)
memory usage: 5.0+ MB


In [20]:
df_ventas.head()

,Marca,Modelo,Version,Precio,Precio_financiado,Combustible,Anyo,Kilometros,Potencia,tipo_cambio,Vendedor_profesional,Vendedor,Provincia,Comunidad_Autonoma
0,SEAT,Toledo,SEAT Toledo 4p.,950,NaN,Diésel,2000.0,227000,NaN,Manual,False,0f4bb8455d27349b8273109b66a847f3,Navarra,Navarra
1,CITROEN,C1,CITROEN C1 PureTech 60KW 82CV Feel 5p.,6200,NaN,Gasolina,2017.0,50071,82.0,Manual,True,Autos Raymara,Tenerife,Canarias
2,FORD,Transit Connect,FORD Transit Connect Van 1.5 TDCi 100cv Ambien...,7851,7024.0,Diésel,2016.0,103000,100.0,Manual,True,Auto 96,Barcelona,Catalunya
3,VOLKSWAGEN,Caravelle,VOLKSWAGEN Caravelle Largo 2.0 TDI 140 Comfort...,19426,NaN,Diésel,2014.0,120000,140.0,Manual,True,Inniauto,Navarra,Navarra
4,FORD,Transit,FORD Transit 350 96kW L4 Ambiente Propulsion T...,22850,22800.0,Diésel,2017.0,107000,130.0,Manual,True,"Autofleet España,s.l",Sevilla,Andalucía


# Compruebo si los nulos de Modelo se puede sustituir por valores fiables

Para ello muestro el valor de la columna versión (que si que tiene todos los
datos) y compruebo si aparece la información que falta

In [21]:
df_ventas.loc[df_ventas.Modelo.isnull(),['Version','Modelo','Combustible','Potencia']]

,Version,Modelo,Combustible,Potencia
1330,SEAT Cordoba 5p.,NaN,Diésel,NaN
25574,SEAT Toledo 4p.,NaN,Diésel,NaN
31826,SEAT,NaN,Diésel,NaN
32831,SEAT Leon 5p.,NaN,Diésel,NaN
37683,SEAT Toledo 1.9 TDI STELLA 110CV 4p.,NaN,Diésel,110.0


In [8]:
acentos = {'á':'a','é':'e','í':'i','ó':'o','ú':'u',
'Á':'A','É':'E','Í':'I','Ó':'O','Ú':'U'}

In [6]:
def quitar_acentos(dic, df, columna):
    for k,v in dic.items():
        df[columna] = df[columna].str.replace(k,v)


In [9]:
quitar_acentos(acentos,df_ventas,'Modelo')

In [11]:
quitar_acentos(acentos,df_ventas,'Version')

quitar_acentos(acentos,df_ventas,'Version')

In [23]:
for k,v in acentos.items():
    df_ventas['Modelo'] = df_ventas.Modelo.str.replace(k,v)

In [12]:
df_ventas.Version[df_ventas.Modelo == 'Cordoba']

198            SEAT Cordoba 1.4 16V 75 CV TOP 4p.
780                             SEAT Cordoba  5p.
1312         SEAT Cordoba 1.9 TDI SPORT 110CV 2p.
2069                            SEAT Cordoba  5p.
2316                            SEAT Cordoba  4p.
                           ...                   
47711        SEAT Cordoba 1.9 TDI STELLA 90CV 4p.
47712    SEAT Cordoba 1.9 TDI 100 CV STYLANCE 4p.
47837                           SEAT Cordoba  5p.
48332                                        SEAT
49086                           SEAT Cordoba  5p.
Name: Version, Length: 77, dtype: object

In [28]:
def clean_modelo(df,columna_ref,columna_modif):
    for vers in df[columna_ref][df[columna_modif].isnull()]:
        mod = vers.split()
        for i in mod:
            if i in df[columna_modif].unique():
                df[columna_modif][df[columna_ref] == vers] = i


In [27]:
clean_modelo(df_ventas,'Version','Modelo');

['SEAT']


In [25]:
for vers in df_ventas.Version[df_ventas.Modelo.isnull()]:
    mod = vers.split()
    print(mod)
    for i in mod:
        if i in df_ventas.Modelo.unique():
            df_ventas.Modelo[df_ventas.Version == vers] = i

['SEAT', 'Cordoba', '5p.']
['SEAT', 'Toledo', '4p.']
['SEAT']
['SEAT', 'Leon', '5p.']
['SEAT', 'Toledo', '1.9', 'TDI', 'STELLA', '110CV', '4p.']


C:\Users\saula\AppData\Local\Temp\ipykernel_20272\3363422952.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas.Modelo[df_ventas.Version == vers] = i


In [26]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marca                 49998 non-null  object 
 1   Modelo                49999 non-null  object 
 2   Version               50000 non-null  object 
 3   Precio                50000 non-null  int64  
 4   Precio_financiado     23563 non-null  float64
 5   Combustible           49954 non-null  object 
 6   Anyo                  49998 non-null  float64
 7   Kilometros            50000 non-null  int64  
 8   Potencia              41472 non-null  float64
 9   tipo_cambio           49889 non-null  object 
 10  Vendedor_profesional  50000 non-null  bool   
 11  Vendedor              50000 non-null  object 
 12  Provincia             49994 non-null  object 
 13  Comunidad_Autonoma    50000 non-null  object 
dtypes: bool(1), float64(3), int64(2), object(8)
memory usage: 5.0+ MB


In [27]:
df_ventas.drop(columns='tipo_cambio',inplace=True)

In [28]:
df_ventas.columns

Index(['Marca', 'Modelo', 'Version', 'Precio', 'Precio_financiado',
       'Combustible', 'Anyo', 'Kilometros', 'Potencia', 'Vendedor_profesional',
       'Vendedor', 'Provincia', 'Comunidad_Autonoma'],
      dtype='object')

In [29]:

df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marca                 50000 non-null  object 
 1   Modelo                49999 non-null  object 
 2   Version               50000 non-null  object 
 3   Precio                50000 non-null  int64  
 4   Precio_financiado     23563 non-null  float64
 5   Combustible           49954 non-null  object 
 6   Anyo                  50000 non-null  float64
 7   Kilometros            50000 non-null  int64  
 8   Potencia              41472 non-null  float64
 9   Vendedor_profesional  50000 non-null  bool   
 10  Vendedor              50000 non-null  object 
 11  Provincia             49994 non-null  object 
 12  Comunidad_Autonoma    50000 non-null  object 
dtypes: bool(1), float64(3), int64(2), object(7)
memory usage: 4.6+ MB


# Nulos de Potencia

In [3]:
df_pot_null = df_ventas.loc[df_ventas.Potencia.isnull(),['Modelo','Combustible']]

In [4]:
df_pot_null.reset_index(inplace=True,drop=True)

In [5]:
df_pot_null.is_copy = False

In [ ]:
def complete_colum_null(df, column_ref_1,column_ref_2,column_null):
    df_aux = df.loc[df[column_null].isnull(),[column_ref_1,column_ref_2]]
    for mod in df_aux[column_ref_1].value_counts().index:
        for com in df_aux[column_ref_2].value_counts().index:
            df_ventas[column_null][
                (df_ventas[column_null].isnull()) &
                (df_ventas[column_ref_1] == mod) &
                (df_ventas[column_ref_2] == com)] = df_ventas[column_null][
                    (df_ventas[column_null].notnull()) &
                    (df_ventas[column_ref_1] == mod) &
                    (df_ventas[column_ref_2] == com)].mean();


In [4]:
for mod in df_pot_null.Modelo.value_counts().index:
    for com in df_pot_null.Combustible.value_counts().index:
        df_ventas.Potencia[
            (df_ventas.Potencia.isnull()) &
             (df_ventas.Modelo == mod) &
              (df_ventas.Combustible == com)] = df_ventas.Potencia[
                  (df_ventas.Potencia.notnull()) &
                   (df_ventas.Modelo == mod) &
                    (df_ventas.Combustible == com)].mean();

C:\Users\saula\AppData\Local\Temp\ipykernel_18768\669084117.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas.Potencia[
C:\Users\saula\AppData\Local\Temp\ipykernel_18768\669084117.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas.Potencia[
C:\Users\saula\AppData\Local\Temp\ipykernel_18768\669084117.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas.Potencia[
C:\Users\saula\AppData\Local\Temp\ipykernel

AttributeError: 'float' object has no attribute 'copy'

In [6]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marca                 49998 non-null  object 
 1   Modelo                49995 non-null  object 
 2   Version               50000 non-null  object 
 3   Precio                50000 non-null  int64  
 4   Precio_financiado     23563 non-null  float64
 5   Combustible           49954 non-null  object 
 6   Anyo                  49998 non-null  float64
 7   Kilometros            50000 non-null  int64  
 8   Potencia              49744 non-null  float64
 9   tipo_cambio           49889 non-null  object 
 10  Vendedor_profesional  50000 non-null  bool   
 11  Vendedor              50000 non-null  object 
 12  Provincia             49994 non-null  object 
 13  Comunidad_Autonoma    50000 non-null  object 
dtypes: bool(1), float64(3), int64(2), object(8)
memory usage: 5.0+ MB


In [6]:
df_def = df_ventas[df_ventas.Potencia.notnull()]

In [7]:
df_def.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41472 entries, 1 to 49999
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marca                 41472 non-null  object 
 1   Modelo                41471 non-null  object 
 2   Version               41472 non-null  object 
 3   Precio                41472 non-null  int64  
 4   Precio_financiado     23442 non-null  float64
 5   Combustible           41470 non-null  object 
 6   Anyo                  41470 non-null  float64
 7   Kilometros            41472 non-null  int64  
 8   Potencia              41472 non-null  float64
 9   tipo_cambio           41361 non-null  object 
 10  Vendedor_profesional  41472 non-null  bool   
 11  Vendedor              41472 non-null  object 
 12  Provincia             41466 non-null  object 
 13  Comunidad_Autonoma    41472 non-null  object 
dtypes: bool(1), float64(3), int64(2), object(8)
memory usage: 4.5+ MB


In [17]:
df_ventas.Potencia[(df_ventas.Potencia.notnull()) & (df_ventas.Modelo == 'Serie 3') & (df_ventas.Combustible == 'Diésel')].mean()

170.32142857142858

No se pueden completar los nulos de potencia, la versión no indica la potencia

In [31]:
df_ventas.Precio_financiado[df_ventas.Precio_financiado.isnull()] = 0

C:\Users\saula\AppData\Local\Temp\ipykernel_20272\1378615709.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas.Precio_financiado[df_ventas.Precio_financiado.isnull()] = 0


In [32]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marca                 50000 non-null  object 
 1   Modelo                49999 non-null  object 
 2   Version               50000 non-null  object 
 3   Precio                50000 non-null  int64  
 4   Precio_financiado     50000 non-null  float64
 5   Combustible           49954 non-null  object 
 6   Anyo                  50000 non-null  float64
 7   Kilometros            50000 non-null  int64  
 8   Potencia              41472 non-null  float64
 9   Vendedor_profesional  50000 non-null  bool   
 10  Vendedor              50000 non-null  object 
 11  Provincia             49994 non-null  object 
 12  Comunidad_Autonoma    50000 non-null  object 
dtypes: bool(1), float64(3), int64(2), object(7)
memory usage: 4.6+ MB


In [33]:
df_ventas.dropna(how='any',inplace=True)

In [34]:
df_ventas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41464 entries, 1 to 49999
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Marca                 41464 non-null  object 
 1   Modelo                41464 non-null  object 
 2   Version               41464 non-null  object 
 3   Precio                41464 non-null  int64  
 4   Precio_financiado     41464 non-null  float64
 5   Combustible           41464 non-null  object 
 6   Anyo                  41464 non-null  float64
 7   Kilometros            41464 non-null  int64  
 8   Potencia              41464 non-null  float64
 9   Vendedor_profesional  41464 non-null  bool   
 10  Vendedor              41464 non-null  object 
 11  Provincia             41464 non-null  object 
 12  Comunidad_Autonoma    41464 non-null  object 
dtypes: bool(1), float64(3), int64(2), object(7)
memory usage: 4.2+ MB


DataFrame filtrado

In [35]:
df_ventas.to_pickle('df_ventas_filtrado.pkl')

In [74]:
Gasolina = ['PETROL','Petrol','petrol','PETROL                   ','Petrol ','E85']
Diesel = ['DIESEL','Diesel','diesel','DIESEL                   ']
Hibrido_Gasoolina = ['Petrol/Electric','PETROL/ELECTRIC',]
Hibrido_Diesel = ['Diesel/Electric','DIESEL/ELECTRIC']
Electrico = ['ELECTRIC','Electric','Electric                 ','electric']
Gas = ['LPG','LPG                      ','NG-biomethane            ','NG-BIOMETHANE','NG-biomethane']
Hidrogeno = ['Hydrogen','HYDROGEN']
Otros = ['OTHER']

In [75]:
df_CO2.loc[df_CO2.Combustible.isin(Gasolina),'Tipo_Comb']='Gasolina'
df_CO2.loc[df_CO2.Combustible.isin(Diesel),'Tipo_Comb']='Diesel'
df_CO2.loc[df_CO2.Combustible.isin(Hibrido_Gasoolina),'Tipo_Comb']='Hibrido_Gasolina'
df_CO2.loc[df_CO2.Combustible.isin(Hibrido_Diesel),'Tipo_Comb']='Hibrido_Diesel'
df_CO2.loc[df_CO2.Combustible.isin(Electrico),'Tipo_Comb']='Electrico'
df_CO2.loc[df_CO2.Combustible.isin(Gas),'Tipo_Comb']='Gas'
df_CO2.loc[df_CO2.Combustible.isin(Hidrogeno),'Tipo_Comb']='Hidrogeno'
df_CO2.loc[df_CO2.Combustible.isin(Otros),'Tipo_Comb']='Otros'


In [76]:
df_CO2.Tipo_Comb.info()

<class 'pandas.core.series.Series'>
RangeIndex: 4955599 entries, 0 to 4955598
Series name: Tipo_Comb
Non-Null Count    Dtype 
--------------    ----- 
4955524 non-null  object
dtypes: object(1)
memory usage: 37.8+ MB


In [77]:
df_CO2.Combustible[df_CO2.CO2_Wltp_gkm.notnull()].value_counts()

Diesel                       1333
Petrol                        357
Electric                      306
DIESEL                        188
ELECTRIC                      118
DIESEL                         54
PETROL                         52
PETROL                          6
Hydrogen                        2
NG-BIOMETHANE                   1
HYDROGEN                        1
Name: Combustible, dtype: int64

hay muy pocos datos de CO2 sobre WLTC por lo que es dificil extrapolar los valores al resto de vehículos

In [80]:
df_CO2.drop(columns=['Combustible','CO2_Wltp_gkm'],inplace=True)

In [98]:
df_ventas.Potencia = df_ventas.Potencia.apply(lambda x: round(x/1.3596,0))

In [106]:
df_CO2_fil = df_CO2[df_CO2.Pais == 'ES']

In [144]:
df_CO2.Pais[(df_CO2.Modelo =='AURIS')&(df_CO2.Potencia.notnull())].value_counts()

FR    11883
PL     9761
CZ     1302
RO      632
LV      340
SK       67
FI       48
LU       29
BG        2
IT        1
AT        1
Name: Pais, dtype: int64

In [99]:
df_ventas.head()

,Marca,Modelo,Version,Precio,Precio_financiado,Combustible,Anyo,Kilometros,Potencia,Vendedor_profesional,Vendedor,Provincia,Comunidad_Autonoma
1,CITROEN,C1,CITROEN C1 PureTech 60KW 82CV Feel 5p.,6200,0.0,Gasolina,2017.0,50071,60.0,True,Autos Raymara,Tenerife,Canarias
2,FORD,Transit Connect,FORD Transit Connect Van 1.5 TDCi 100cv Ambien...,7851,7024.0,Diésel,2016.0,103000,74.0,True,Auto 96,Barcelona,Catalunya
3,VOLKSWAGEN,Caravelle,VOLKSWAGEN Caravelle Largo 2.0 TDI 140 Comfort...,19426,0.0,Diésel,2014.0,120000,102.0,True,Inniauto,Navarra,Navarra
4,FORD,Transit,FORD Transit 350 96kW L4 Ambiente Propulsion T...,22850,22800.0,Diésel,2017.0,107000,96.0,True,"Autofleet España,s.l",Sevilla,Andalucía
5,PEUGEOT,3008,PEUGEOT 3008 Style 1.2 PureTech 130 SS 5p.,11490,10490.0,Gasolina,2016.0,78665,96.0,True,Flexicar Rivas,Madrid,Comunidad_de_Madrid


In [100]:
df_ventas[df_ventas.Modelo == 'Captur']

,Marca,Modelo,Version,Precio,Precio_financiado,Combustible,Anyo,Kilometros,Potencia,Vendedor_profesional,Vendedor,Provincia,Comunidad_Autonoma
33,RENAULT,Captur,RENAULT Captur Zen TCe 66kW 90CV 5p.,14650,14650.0,Gasolina,2019.0,14000,66.0,True,Ald Automotive,Madrid,Comunidad_de_Madrid
428,RENAULT,Captur,RENAULT Captur S EDITION TCe GPF 110kW 150CV 5p.,18900,17500.0,Gasolina,2019.0,5,110.0,True,Castilla Selection Salamanca,Salamanca,Castilla_y_León
439,RENAULT,Captur,RENAULT Captur Zen dCi 66kW 90CV 5p.,16000,13700.0,Diésel,2019.0,31847,66.0,True,Talleres Escorial,Madrid,Comunidad_de_Madrid
467,RENAULT,Captur,RENAULT Captur Zen Energy TCe 88kW 120CV EDC 5p.,13990,12990.0,Gasolina,2018.0,82000,88.0,True,Carlecar Costa SL,Huelva,Andalucía
1018,RENAULT,Captur,RENAULT Captur Zen Blue dCi 85kW 115CV EDC 5p.,22500,20500.0,Diésel,2020.0,5926,85.0,True,Renault Norte Motor,Asturias,Principado_de_Asturias
...,...,...,...,...,...,...,...,...,...,...,...,...,...
49093,RENAULT,Captur,RENAULT Captur Intens Energy dCi 90 SS eco2 5p.,10500,0.0,Diésel,2013.0,69000,66.0,True,JAYGO .NET,Pontevedra,Galicia
49236,RENAULT,Captur,RENAULT Captur Zen TCe GPF 110kW 150CV EDC 5p.,23000,21500.0,Gasolina,2019.0,9100,110.0,True,Herrero y Lopez,Murcia,Región_de_Murcia
49520,RENAULT,Captur,RENAULT Captur Zen TCe 66kW 90CV 5p.,15990,14490.0,Gasolina,2019.0,7409,66.0,True,Vidal de la Peña Automoviles,Cantabria,Cantabria
49607,RENAULT,Captur,RENAULT Captur Zen TCe 96kW 130CV EDC 5p.,22400,20900.0,Gasolina,2020.0,10150,96.0,True,Rodosa,Pontevedra,Galicia
